In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

from ICARUS.Core.units import calc_mach, calc_reynolds
from ICARUS.Database import DB
from ICARUS.Airfoils.airfoil import Airfoil
from ICARUS.Core.struct import Struct
from ICARUS.Computation.Solvers.solver import Solver
from ICARUS.Computation.Solvers.OpenFoam.files.setup_case import MeshType

# Reynolds And Mach and AoA

In [ ]:
# Assumptions
chord_max: float = 0.18
chord_min: float = 0.11
u_max: float = 30.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_min: float = calc_mach(10, speed_of_sound)
mach_max: float = calc_mach(30, speed_of_sound)
mach = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds = np.logspace(
    start=np.log10(reynolds_min),
    stop=np.log10(reynolds_max),
    num=2,
    base=10,
)
REYN = reynolds_max

# ANGLE OF ATTACK SETUP
aoa_max: float = 2
aoa_min: float = -2
num_of_angles: float = (aoa_max - aoa_min) * 2 + 1
angles = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)
airfoil_name: str = "2418"
airfoil: Airfoil = Airfoil.naca(naca=airfoil_name, n_points=200)

ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}

In [ ]:
print(f"Nominal: Mach {MACH}\tReynolds {REYN}")


In [ ]:
cleaning = False
calcF2W = False
calcOpenFoam = False
calcXFoil = True

# Get Airfoil

In [ ]:
airf: Airfoil = airfoil.naca('4415',n_points=200)
airf.plot(camber=True)

# Foil2Wake

In [ ]:
# if cleaning :
    # airf.clean_results(f2w.remove_results, [airf.REYNDIR, airf.HOMEDIR,angles])


if calcF2W :
    # Import Solver
    from ICARUS.Computation.Solvers.Foil2Wake.f2w_section import Foil2Wake
    f2w_s: Solver = Foil2Wake()

    # Import Analysis
    analysis: str = f2w_s.get_analyses_names()[0]  # Run
    f2w_s.select_analysis(analysis)

    # Get Options
    options: Struct = f2w_s.get_analysis_options(verbose=True)
    solver_parameters: Struct = f2w_s.get_solver_parameters()

    # Set Options
    options.airfoil = airfoil
    options.reynolds = reynolds
    options.mach = MACH
    options.f_trip_low = ftrip_low
    options.f_trip_upper = ftrip_up
    options.angles = angles

    # RUN
    f2w_s.set_analysis_options(options)
    f2w_s.print_analysis_options()
    f2w_s.execute()

    # Get Results
    polars = f2w_s.get_results()


# Xfoil

In [ ]:
if calcXFoil:
    airfoils = [airf.name]#["0008"]
    from ICARUS.Computation.Solvers.Xfoil.xfoil import Xfoil 

    for airfoil in airfoils:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get airfoil
        airf: Airfoil = Airfoil.naca(airfoil, n_points=200)
        # airf.plotAirfoil()
        xfoil: Solver = Xfoil()

        # Import Analysis
        analysis: str = xfoil.get_analyses_names()[0]  # Run
        xfoil.select_analysis(analysis)

        # Get Options
        options = xfoil.get_analysis_options(verbose=True)
        solver_parameters = xfoil.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.reynolds = reynolds
        options.mach = MACH
        options.min_aoa = -5
        options.max_aoa = 5
        options.aoa_step = 1
        xfoil.print_analysis_options()

        # Set Solver Options
        solver_parameters.max_iter = 400
        solver_parameters.Ncrit = 1e-3
        solver_parameters.xtr = (0.1, 0.1)
        solver_parameters.print = False
        # xfoil.print_solver_options()
        
        # RUN
        xfoil.execute()


# Mapflow

# OpenFoam (NOT HIGH MACH)

In [ ]:
if calcOpenFoam:
    airfoils: list[str] = [airf.name]#["0008"]
    from ICARUS.Computation.Solvers.OpenFoam.open_foam import OpenFoam

    for airfoil in airfoils:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: Airfoil = Airfoil.naca(airfoil, n_points=200)
        # airf.plot()
        open_foam: Solver = OpenFoam()

        # Import Analysis
        analysis: str = open_foam.get_analyses_names()[0]  # Run
        open_foam.select_analysis(analysis)

        # Get Options
        options = open_foam.get_analysis_options(verbose=True)
        solver_parameters = open_foam.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.angles = angles
        options.reynolds = REYN
        options.mach = MACH
        open_foam.print_analysis_options()

        # Set Solver Options
        solver_parameters.mesh_type = MeshType.structAirfoilMesher
        solver_parameters.max_iterations = 20
        solver_parameters.silent = False
        
        # RUN
        open_foam.set_analysis_options(options)
        open_foam.print_analysis_options()
        open_foam.execute()